In [3]:
from src.filters.f_cytotox import CytotoxicityFilter
from src.filters.f_distribution import DistributionFilter

from pathlib import Path
import pandas as pd
import numpy as np

**FILTERING THE GENERATED DB FOR MUTANT SELECTION**

In [2]:
CTT_PATH = Path('src/models/classificators/RFC_esm.pkl')
DIST_DATA_PATH = Path('src/data/acp_trainset.csv')

In [3]:
filters = []
filters.append(CytotoxicityFilter(CTT_PATH, 'mps'))
filters.append(DistributionFilter(DIST_DATA_PATH))

/Users/macbook/miniconda3/envs/acp_gen/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/macbook/miniconda3/envs/acp_gen/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


initializing the embedding model


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
DB = pd.read_csv('src/data/mut_DBs/rnn_acp_mp/DB.csv',index_col=False)
sequences = list(DB['sequence'].to_numpy())

sequences

['AAAANGPRRR',
 'AAAARRRR',
 'AAAARRRRRRRR',
 'AAAARRRRWRRRARRAARR',
 'AAAIINLTSL',
 'AAAIRGC',
 'AAAIRILNRL',
 'AAAIRLNAL',
 'AADILTF',
 'AAEIGDKSWLY',
 'AAEIGDRSYL',
 'AAEIGRKDLLL',
 'AAGIGILTL',
 'AAGIGILTV',
 'AAKAIAKAIAALAKLAKKI',
 'AAKIAAKIAKKIAKSIAKFL',
 'AAKIIADILKNLLR',
 'AAKIIAKIAKKI',
 'AAKIIAKIAKKIA',
 'AAKIIAKIAKKIF',
 'AAKIIAKIARNILKGA',
 'AAKIIANFLPQIACIARTC',
 'AAKIILNAK',
 'AAKIILNDLKKIALNWLRA',
 'AAKIILNPKFCIAFC',
 'AAKIILNPKFRCKAAFC',
 'AAKIINNLKAYLAKLF',
 'AAKKWAKAKWAKAKKAA',
 'AAKKWAKAKWAKAKKWA',
 'AAKKWAKAKWAKAKKWAKAA',
 'AAKLIINALTF',
 'AALGIPGIC',
 'AALNWKPLLKALAAKFL',
 'AALPLW',
 'AALPLWACKISKQC',
 'AALPLWAPQLGKKILAALNSIF',
 'AAMAI',
 'AANPILCL',
 'AARAWRAWFRAARRFLRRAGRAFLRRASRALL',
 'AARCILCL',
 'AARCILS',
 'AARIILRARFR',
 'AARIILRDRFR',
 'AARIILRERFR',
 'AARIILRFRFR',
 'AARIILRGRFR',
 'AARIILRIRF',
 'AARIILRIRFR',
 'AARIILRIRRR',
 'AARIILRLFRF',
 'AARIILRLRFR',
 'AARIILRNRFR',
 'AARIILRRRFR',
 'AARIILRTRFR',
 'AARIILRWFR',
 'AARIILRWFRF',
 'AARIILRWRFR',
 'AA

In [5]:
for filter in filters:
    DB = filter.filter_sequences(DB)

DB.to_csv('src/data/mut_DBs/acp_mp/DB.csv',index=False)

generating sequence embeddings


100%|██████████| 326/326 [00:25<00:00, 12.83it/s]


In [25]:
### remove already existing ACPs
OG = pd.read_csv('src/data/known_acps/acp_mp.csv',index_col=False)
DB_filtered = DB[~DB['sequence'].isin(set(OG['sequence']))]
DB_filtered.to_csv('src/data/mut_DBs/rnn_acp_mp/DB_filtered.csv', index=False)

**GENERATE ESM-EMBEDDS FOR SEQ DB**

In [22]:
from src.utils.gen_esm_embedds import Embedder

sequences = list(DB_filtered['sequence'].to_numpy())

embedder = Embedder('mps')
embeddings = embedder.get_embeddings(sequences)
np.save('src/data/mut_DBs/rnn_acp_mp/esm_embedds_filtered.npy',embeddings)

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 304/304 [00:25<00:00, 11.92it/s]


In [12]:
seq = sequences[0]
seq
single_embedds = embedder.get_embeddings([seq],bs=1)

100%|██████████| 1/1 [00:00<00:00,  1.99it/s]


**TESTING PROGEN2_SMALL GENERATOR**

In [1]:
from src.generators.genProGen import GenProGen
from pathlib import Path

/Users/macbook/miniconda3/envs/acp_gen/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_path = Path('/Users/macbook/Documents/RoVa/Masters/ACP_designer/ACP_designer_WEB_repo/ACP-designer-WEB/src/models/generative/progen2-small-ACP-MP')

progen_acp = GenProGen(model_path)

In [4]:
progen_acp.generate_sequences(10)

,sequence,label
0,GFIRKLVHTMWN,GEN
1,GFLPRAEVKHDMINWCT,GEN
2,GLFSKIWHVANDMTER,GEN
3,GRFLHWAQNIKPDMTS,GEN
4,HGKFRYLSDNVAEIWQTMC,GEN
5,HLARVISETGKPY,GEN
6,HRWLDQIKVTS,GEN
7,KCRLTIVSGPDFEQYHANW,GEN
8,KISVWFCPDTARNMYL,GEN
9,YVLKGDIFNRAPWH,GEN
